<h1 align="center">Primeiros passos com a Prompt Engineering</h1>

Data Scientist Jr.: Karina Gonçalves Soares

Links de estudo:

* [Prompt-Engineering-Guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-lecture.ipynb)
* [O que é Engenharia de Prompt...](https://www.alura.com.br/artigos/engenharia-prompt)


por DAIR.AI | Elvis Saravia

    Este caderno contém exemplos e exercícios para aprender sobre Prompt Engineering.

    Usaremos as APIs OpenAI para todos os exemplos. Estou usando as configurações padrão de temperature = 0,7 e top-p = 1

1. Noções básicas de Prompt Engineering

* Objetivos

    * Carregue as bibliotecas
    * Revise o formato
    * Cubra os prompts básicos
    * Revise casos de uso comuns

Abaixo estamos carregando as bibliotecas, utilitários e configurações necessárias.

In [1]:
import openai # Permite interação com modelos de linguagem treinados pela OpenAI
import os
import IPython # Recursos para execução interativa 
from langchain.llms import OpenAI
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv()) # read lical ,env file
key_openai = os.environ['OPENAI_API_KEY']

In [3]:
def set_open_params(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,  
):
    """ definir parâmetros openai"""
    
    openai_params = {}
    
    
    openai_params["model"] = model
    openai_params["temperature"] = temperature
    openai_params["max_tokens"] = max_tokens
    openai_params["top_p"] = top_p
    openai_params["frequency_penalty"] = frequency_penalty
    openai_params["presence_penalty"] = presence_penalty
    return openai_params

def get_completion(params, messages):
    """ obter resposta da API openai"""
    
    
    response = openai.chat.completions.create(
        model = params['model'],
        messages = messages,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

    
    

Exemplo de prompt básico:

In [4]:
params = set_open_params()

prompt = "qual é a cor do céu?"

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = get_completion(params, messages)

In [5]:
response.choices[0].message.content

'A cor do céu pode variar dependendo da hora do dia e das condições meteorológicas. Geralmente, o céu é azul durante o dia e pode mudar para tons de laranja, rosa, roxo ou vermelho durante o nascer ou pôr do sol. Durante a noite, o céu pode parecer preto ou escuro, mas em áreas com pouca poluição luminosa, é possível ver estrelas e a Via Láctea, que dão ao céu uma tonalidade azul escura ou roxa.'

Experimente com temperaturas diferentes para comparar os resultados:

In [6]:
params = set_open_params(temperature=0)
response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)

A cor do céu pode variar dependendo da hora do dia e das condições climáticas. Geralmente, durante o dia, o céu é azul, mas pode apresentar tons de laranja, rosa, roxo e vermelho durante o nascer e o pôr do sol. Durante a noite, o céu pode ser preto ou escuro, mas também pode apresentar tons de azul escuro, roxo e prateado, dependendo da iluminação da lua e das estrelas.

### 1.1 Resumo do Texto

In [12]:
params = set_open_params(temperature=0)

prompt ="""Os antibióticos são um tipo de medicamento usado para tratar infecções bacterianas.\n
 Eles atuam matando as bactérias ou impedindo-as de se reproduzirem, permitindo que o sistema imunológico do corpo combata a infecção.\n
 Os antibióticos são geralmente tomados por via oral na forma de comprimidos, cápsulas ou soluções líquidas, ou às vezes administrados por via intravenosa.\n
 Eles não são eficazes contra infecções virais e seu uso inadequado pode levar à resistência aos antibióticos."""
 
 
messages = [
    {
        "role": "user",
        "content": prompt
    }
 ]

response = get_completion(params, messages)
IPython.display.Markdown(response.choices[0].message.content)



É importante seguir as instruções do médico ao tomar antibióticos, incluindo completar todo o curso de tratamento, mesmo que os sintomas melhorem antes. Interromper o tratamento prematuramente pode resultar em uma infecção recorrente ou resistência aos antibióticos.

Além disso, é importante não compartilhar antibióticos com outras pessoas, pois o medicamento pode não ser apropriado para a condição delas e pode causar efeitos colaterais indesejados.

É essencial informar ao médico sobre qualquer alergia a antibióticos ou outros medicamentos, bem como qualquer outra condição de saúde que possa afetar o uso seguro dos antibióticos.

Em resumo, os antibióticos são uma ferramenta importante no tratamento de infecções bacterianas, mas seu uso deve ser cuidadosamente monitorado e seguido de acordo com as orientações médicas para garantir a eficácia e evitar problemas de resistência.